In [117]:
import json
import networkx as nx
import random
import matplotlib.pyplot as plt
import math
log = math.log2
from sympy import *

In [118]:
channel_graph = nx.Graph()

In [198]:
channel_graph.add_edge("S","A",capacity=18)
channel_graph.add_edge("S","C",capacity=5)
channel_graph.add_edge("A","B",capacity=5)
channel_graph.add_edge("A","D",capacity=7)
#cross
channel_graph.add_edge("A","C",capacity=18)
channel_graph.add_edge("C","B",capacity=18)
channel_graph.add_edge("C","D",capacity=15)
channel_graph.add_edge("C","A",capacity=18)
#cross
channel_graph.add_edge("B","D",capacity=18)
channel_graph.add_edge("B","R",capacity=18)
channel_graph.add_edge("D","R",capacity=18)

In [199]:
def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def get_sid(src,dest):
    return "{}:{}".format(src,dest)

In [200]:
path_num = 0
channels = {}
for src,dest in channel_graph.edges():
    channels[get_sid(src,dest)]=[]
    channels[get_sid(dest,src)]=[]

print(channels)

used_variables = []
for path in nx.all_simple_paths(channel_graph,"S","R",cutoff=6):
    print(path_num, path)
    var = symbols("a{}".format(path_num))
    for src, dest in next_hop(path):
        sid = get_sid(src,dest)
        channels[sid].append(var)
    used_variables.append(var)
    path_num+=1
channels       
        

{'S:A': [], 'A:S': [], 'S:C': [], 'C:S': [], 'A:B': [], 'B:A': [], 'A:D': [], 'D:A': [], 'A:C': [], 'C:A': [], 'C:B': [], 'B:C': [], 'C:D': [], 'D:C': [], 'B:R': [], 'R:B': [], 'B:D': [], 'D:B': [], 'D:R': [], 'R:D': []}
0 ['S', 'A', 'B', 'C', 'D', 'R']
1 ['S', 'A', 'B', 'R']
2 ['S', 'A', 'B', 'D', 'R']
3 ['S', 'A', 'D', 'C', 'B', 'R']
4 ['S', 'A', 'D', 'R']
5 ['S', 'A', 'D', 'B', 'R']
6 ['S', 'A', 'C', 'B', 'R']
7 ['S', 'A', 'C', 'B', 'D', 'R']
8 ['S', 'A', 'C', 'D', 'R']
9 ['S', 'A', 'C', 'D', 'B', 'R']
10 ['S', 'C', 'B', 'A', 'D', 'R']
11 ['S', 'C', 'B', 'R']
12 ['S', 'C', 'B', 'D', 'R']
13 ['S', 'C', 'D', 'A', 'B', 'R']
14 ['S', 'C', 'D', 'R']
15 ['S', 'C', 'D', 'B', 'R']
16 ['S', 'C', 'A', 'B', 'R']
17 ['S', 'C', 'A', 'B', 'D', 'R']
18 ['S', 'C', 'A', 'D', 'R']
19 ['S', 'C', 'A', 'D', 'B', 'R']


{'A:B': [a0, a1, a2, a13, a16, a17],
 'A:C': [a6, a7, a8, a9],
 'A:D': [a3, a4, a5, a10, a18, a19],
 'A:S': [],
 'B:A': [a10],
 'B:C': [a0],
 'B:D': [a2, a7, a12, a17],
 'B:R': [a1, a3, a5, a6, a9, a11, a13, a15, a16, a19],
 'C:A': [a16, a17, a18, a19],
 'C:B': [a3, a6, a7, a10, a11, a12],
 'C:D': [a0, a8, a9, a13, a14, a15],
 'C:S': [],
 'D:A': [a13],
 'D:B': [a5, a9, a15, a19],
 'D:C': [a3],
 'D:R': [a0, a2, a4, a7, a8, a10, a12, a14, a17, a18],
 'R:B': [],
 'R:D': [],
 'S:A': [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9],
 'S:C': [a10, a11, a12, a13, a14, a15, a16, a17, a18, a19]}

In [201]:
def channel_probability(capacity,expression):
    return (capacity+1-expression)/(capacity+1)

def create_target_function():
    res = 1
    for sid,variables in channels.items():
        expression=0
        for variable in variables:
            expression+=variable
        src,dest = sid.split(":")
        eq = channel_probability(channel_graph[src][dest]["capacity"],expression)
        #print(eq)
        res*=eq
    return res

f = create_target_function()

In [202]:
print(f)

1.0*(-a0/19 + 1)*(-a10/6 + 1)*(-a13/8 + 1)*(-a3/16 + 1)*(-a12/19 - a17/19 - a2/19 - a7/19 + 1)*(-a15/19 - a19/19 - a5/19 - a9/19 + 1)*(-a16/19 - a17/19 - a18/19 - a19/19 + 1)*(-a6/19 - a7/19 - a8/19 - a9/19 + 1)*(-a0/6 - a1/6 - a13/6 - a16/6 - a17/6 - a2/6 + 1)*(-a0/16 - a13/16 - a14/16 - a15/16 - a8/16 - a9/16 + 1)*(-a10/8 - a18/8 - a19/8 - a3/8 - a4/8 - a5/8 + 1)*(-a10/19 - a11/19 - a12/19 - a3/19 - a6/19 - a7/19 + 1)*(-0.0526315789473684*a0 - 0.0526315789473684*a1 - 0.0526315789473684*a2 - 0.0526315789473684*a3 - 0.0526315789473684*a4 - 0.0526315789473684*a5 - 0.0526315789473684*a6 - 0.0526315789473684*a7 - 0.0526315789473684*a8 - 0.0526315789473684*a9 + 1.0)*(-a0/19 - a10/19 - a12/19 - a14/19 - a17/19 - a18/19 - a2/19 - a4/19 - a7/19 - a8/19 + 1)*(-a1/19 - a11/19 - a13/19 - a15/19 - a16/19 - a19/19 - a3/19 - a5/19 - a6/19 - a9/19 + 1)*(-a10/6 - a11/6 - a12/6 - a13/6 - a14/6 - a15/6 - a16/6 - a17/6 - a18/6 - a19/6 + 1)


In [203]:
cache = set()
def gen(x,a):
    s = sum(x)
    if s == a:
        string = ",".join(str(y) for y in x)
        if string not in cache:
            cache.add(string)
            yield(x)
    if s < a:
        for i in range(len(x)):
            x[i]+=1
            yield from gen(x,a)
            x[i]-=1


In [204]:
x = [0 for var in used_variables]
#x = [0,0,0]
a = 3

print("start")
m = 0
b = None
for res in gen(x,a):
    subs=dict(zip(used_variables,res))
    e = (f).evalf(subs = subs)
    #print(e)
    if e>m:
        m=e
        b = res.copy()
        print(m,res)

print(m,b)

start
0.242600962239394 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.279417745413249 [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.289528255938070 [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.323856733245857 [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.328456424803693 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.369817604223418 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.393643388249016 [1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.416387228458959 [0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.422648690541049 [0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.446129173348885 [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.467138026387475 [0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.472699193368278 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.480484827141403 [0, 0, 0, 0, 2, 0, 1, 0, 0, 

In [ ]:
subs=dict(zip(variables[:-1],res[:-1]))
print((f).evalf(subs = subs))

In [81]:
print(x)

[]


In [105]:
for res in gen([0,0,0],3):
    print(res)

[3, 0, 0]
[2, 1, 0]
[2, 0, 1]
[1, 2, 0]
[1, 1, 1]
[1, 0, 2]
[0, 3, 0]
[0, 2, 1]
[0, 1, 2]
[0, 0, 3]


In [113]:
def create_and_solve_lagrange_problem(f,variables,amt):
    partial_derrivatives = [diff(f,var) for var in variables]
    l = symbols("l")
    non_linear_equations = [Eq(p,l) for p in partial_derrivatives]
    split = 0
    for var in variables:
        split=split+var
    variables.append(l)
    non_linear_equations.append(Eq(split,amt))
    start = [amt/(len(variables)-1) for var in variables[:-1]]
    start.append(1)
    print(non_linear_equations,split, start)
    print((f).evalf(subs=dict(zip(variables[:-1],start))))
    res = nsolve(non_linear_equations,variables,start, simplify=False)
    print(amt, res)
    subs=dict(zip(variables[:-1],res[:-1]))
    print((f).evalf(subs = subs))

In [62]:
print(used_variables)
create_and_solve_lagrange_problem(f,used_variables,16)

[a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11]
[Eq(0.0909090909090909*(a0/11 + 1)*(a2/11 + 1)*(a6/11 + 1)*(a8/11 + 1)*(a10/11 + a11/11 + 1)*(a4/11 + a5/11 + 1)*(a0/11 + a1/11 + a10/11 + a8/11 + 1)*(a0/11 + a5/11 + a8/11 + a9/11 + 1)*(a11/11 + a2/11 + a3/11 + a6/11 + 1)*(a2/11 + a4/11 + a6/11 + a7/11 + 1)*(0.0909090909090909*a0 + 0.0909090909090909*a1 + 0.0909090909090909*a2 + 0.0909090909090909*a3 + 0.0909090909090909*a4 + 0.0909090909090909*a5 + 1.0)*(a1/11 + a10/11 + a2/11 + a4/11 + a7/11 + a8/11 + 1)*(a10/11 + a11/11 + a6/11 + a7/11 + a8/11 + a9/11 + 1) + 0.0909090909090909*(a0/11 + 1)*(a2/11 + 1)*(a6/11 + 1)*(a8/11 + 1)*(a10/11 + a11/11 + 1)*(a4/11 + a5/11 + 1)*(a0/11 + a1/11 + a10/11 + a8/11 + 1)*(a0/11 + a5/11 + a8/11 + a9/11 + 1)*(a11/11 + a2/11 + a3/11 + a6/11 + 1)*(a2/11 + a4/11 + a6/11 + a7/11 + 1)*(a0/11 + a11/11 + a3/11 + a5/11 + a6/11 + a9/11 + 1)*(a1/11 + a10/11 + a2/11 + a4/11 + a7/11 + a8/11 + 1)*(a10/11 + a11/11 + a6/11 + a7/11 + a8/11 + a9/11 + 1) + 0.090909090909

ZeroDivisionError: matrix is numerically singular